# Installation of Libraries

If you use our code, please cite:

@misc{2024<br>
  title = {Mistral -7b hosting},<br>
  author = {Hamza Farooq, Darshil Modi, Kanwal Mehreen, Nazila Shafiei},<br>
  keywords = {LLM Hosting, Mistral-7b},<br>
  year = {2024},<br>
  copyright = {MIT, non-exclusive license}<br>
}


In [1]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [2]:
!pip install git+https://github.com/huggingface/transformers torch accelerate bitsandbytes langchain

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-tzcps6os
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-tzcps6os
  Resolved https://github.com/huggingface/transformers to commit 0ade1caa356dce6b70ef8293addeb0898f177206
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.7 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.48.0.dev0-py3-none-any.whl size=10283310 sha256=16cb79592415a0345c308fdbf453d35b0f0e61a2a29ceb2b03aeb2a629b70f3c
  Stored in directory: /tmp/pip-ephem-wheel-cache-nks6_jbh/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Succes

In [3]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [6]:
!pip install pyngrok --quiet
!pip install fastapi nest-asyncio --quiet
!pip install uvicorn --quiet
!pip install langchain-community --quiet
!pip install -U langchain-huggingface --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.5/411.5 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.7 MB/s eta 0:00:00


# Setup

Get you Hugging Face Token, [here](https://huggingface.co/settings/tokens)

In [5]:
from google.colab import userdata
HUGGING_FACE_TOKEN = userdata.get('HUGGING_FACE_TOKEN')

In [7]:
!huggingface-cli login --token $HUGGING_FACE_TOKEN

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `MAVEN_COURSE_WRITE` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `MAVEN_COURSE_WRITE`


In [8]:
# Import libraries
from transformers import AutoModelForCausalLM, AutoTokenizer , BitsAndBytesConfig
import transformers
import torch
from langchain_huggingface import HuggingFacePipeline
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from threading import Thread

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [9]:
!nvidia-smi

Thu Dec 19 19:40:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [10]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

# Basic Prompt

**Note:** If you are on windows then bitsandbytes won't work. You can counter this with bitsandbytes-windows but that is and older version that only supports 8-bit quantization.

In [14]:
# Define the model ID for the desired model
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

# Define the quantization configuration for the model
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load the model using the model ID and quantization configuration
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, device_map='auto')

# Load the tokenizer associated with the model
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [15]:
vegetarian_recipe_prompt = """### Instruction: Act as a gourmet chef.
what is the best chicken dish for large groups

 ### Answer:
 """
prompt_template = """
### [INST]
Instruction: Act as a gourmet chef.


### QUESTION:
what is the best chicken dish for large groups

[/INST]
 """

# Encode the instruction using the tokenizer
encoded_instruction = tokenizer(prompt_template, return_tensors="pt", add_special_tokens=True)

# Move the encoded instruction to the appropriate device (e.g., GPU)
model_inputs = encoded_instruction.to(device)

# Generate text based on the model inputs
generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

# Decode the generated text
decoded = tokenizer.batch_decode(generated_ids)

# Print the decoded output
print(decoded)

['<s> \n### [INST]\nInstruction: Act as a gourmet chef.\n\n\n### QUESTION:\nwhat is the best chicken dish for large groups\n\n[/INST]\n \nA great chicken dish for a large group is roast chicken. This classic dish is a must-have at any large gathering and is easy to prepare. You can roast the chicken in the oven with your favorite herbs and spices, serve it with a side of roasted vegetables or mashed potatoes. Another option is a chicken fajita, which is a great option for groups of people who like some heat. Simply cook the chicken with bell peppers and onions, serve it in a warm tortilla with salsa and guacamole. Chicken stir-fry is also a great option for a large group. Cook the chicken with your favorite vegetables, such as broccoli and bell peppers, and serve it with rice.</s>']


In [16]:
response = decoded[0].split("[/INST]")[1]
print((response.split('</s>')[0]).strip())

A great chicken dish for a large group is roast chicken. This classic dish is a must-have at any large gathering and is easy to prepare. You can roast the chicken in the oven with your favorite herbs and spices, serve it with a side of roasted vegetables or mashed potatoes. Another option is a chicken fajita, which is a great option for groups of people who like some heat. Simply cook the chicken with bell peppers and onions, serve it in a warm tortilla with salsa and guacamole. Chicken stir-fry is also a great option for a large group. Cook the chicken with your favorite vegetables, such as broccoli and bell peppers, and serve it with rice.


# Creating LLM Chain to run a RAG prompt


In [17]:

text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",  # Specify the task as text generation
    temperature=0.3,  # Temperature parameter for controlling randomness in sampling
    repetition_penalty=1.1,  # Repetition penalty parameter to avoid repeating tokens
    return_full_text=True,  # Flag to return full text instead of a list of generated tokens
    max_new_tokens=1000,  # Maximum number of tokens to generate
    do_sample=True  # Flag to use sampling during text generation
)

prompt_template = """
### [INST]
Instruction: I will ask you a QUESTION and give you a CONTEXT and you will respond with an answer easily understandable.

### CONTEXT:
{context}

### QUESTION:
{question}

[/INST]
 """

# Create HuggingFacePipeline object wrapping the text generation pipeline
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Create prompt object from the prompt template with input variables as context and question
prompt = PromptTemplate(
    input_variables=["context", "question"],  # Specify input variables for the prompt
    template=prompt_template,  # Specify the template for the prompt
)

# Create language model chain (llm_chain) with HuggingFacePipeline and prompt
llm_chain = prompt | mistral_llm | StrOutputParser()


Device set to use cuda:0


In [18]:
prompt_base = """
### [INST]
Instruction: Act as a gourmet chef.


### QUESTION:
what is the best chicken dish for large groups

[/INST]
 """


In [19]:
response=text_generation_pipeline(prompt_base)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [20]:
response

[{'generated_text': '\n### [INST]\nInstruction: Act as a gourmet chef.\n\n\n### QUESTION:\nwhat is the best chicken dish for large groups\n\n[/INST]\n  As a gourmet chef, I would recommend roasting a whole chicken for a large group gathering. This classic dish is not only delicious but also easy to prepare and serves a crowd. To make it even more special, you can add some herbs and spices to the chicken before roasting, such as rosemary, thyme, garlic, and paprika. You can also serve it with a side of roasted vegetables or a fresh salad to complement the dish. Additionally, you can make sure that the chicken is cooked thoroughly by using a meat thermometer to check the internal temperature.'}]

In [21]:
r=response[0].get('generated_text').split("[/INST]")[1].split('</s>')[0]


In [22]:
print(r.strip())

As a gourmet chef, I would recommend roasting a whole chicken for a large group gathering. This classic dish is not only delicious but also easy to prepare and serves a crowd. To make it even more special, you can add some herbs and spices to the chicken before roasting, such as rosemary, thyme, garlic, and paprika. You can also serve it with a side of roasted vegetables or a fresh salad to complement the dish. Additionally, you can make sure that the chicken is cooked thoroughly by using a meat thermometer to check the internal temperature.


In [23]:
# Input query to LLM
input_text = {
    "context": "Artificial Intelligence (AI) is the stream of data science that can predict future based on past trends!",
    "question": "What is AI ?"
}

# Generate and print LLM response
response = llm_chain.invoke(input_text)

wrapped_text = wrap_text(response)
print(f'The answer is: \n {wrapped_text}')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The answer is: 
 
### [INST]
Instruction: I will ask you a QUESTION and give you a CONTEXT and you will respond with an
answer easily understandable.

### CONTEXT:
Artificial Intelligence (AI) is the stream of data science that can predict future based
on past trends!

### QUESTION:
What is AI ?

[/INST]

Artificial Intelligence (AI) refers to the ability of machines or computer systems to
perform tasks that would normally require human intelligence, such as visual perception,
speech recognition, decision-making, language translation, and learning from experience.
In other words, AI allows machines to reason, learn, and adapt in ways that are similar to
how humans do.


In [24]:
##############
question_rag = "What are the differences in work expectations between generations?"
##############


context_rag = """
'Generational differences in work expectations and values are a significant topic of discussion in organizations today. Specifically, the differences between baby boomers and millennials have garnered attention due to their coexistence in the workplace. These generational differences can impact management dynamics at various levels, including operational, team, and strategic levels. Understanding these differences and finding ways to navigate them is crucial for creating a harmonious and productive work environment.

1. Contrasting Work Values, Attitudes, and Preferences:
Baby Boomers:
- Work Values: Baby boomers tend to prioritize loyalty, dedication, and hard work. They often value stability and long-term commitment to their organizations.
- Attitudes: Baby boomers may have a more traditional approach to work, valuing hierarchy and authority. They may prefer a structured work environment and clear guidelines.
- Preferences: Baby boomers may prefer face-to-face communication and value personal relationships in the workplace. They may also prioritize financial stability and job security.

Millennials:
- Work Values: Millennials often prioritize work-life balance, personal growth, and purpose-driven work. They seek opportunities for learning and development.
- Attitudes: Millennials may have a more collaborative and inclusive approach to work. They value flexibility, autonomy, and a sense of purpose in their roles.
- Preferences: Millennials are comfortable with technology and may prefer digital communication channels. They value feedback and recognition and seek a positive and engaging work environment.

2. Factors Contributing to Variations:
a) Socioeconomic Factors: Socioeconomic factors such as technological advancements, economic conditions, and societal changes can shape the expectations and values of different generations. For example, baby boomers experienced significant economic growth and stability, while millennials entered the workforce during times of economic uncertainty.
b) Cultural Influences: Cultural factors, including societal norms, values, and beliefs, can also contribute to generational differences. Each generation grows up in a unique cultural context that influences their perspectives on work.
c) Life Experiences: Different life experiences, such as major historical events or technological advancements, can shape the values and attitudes of each generation. For example, baby boomers may have experienced significant social and political changes, while millennials grew up in the digital age.

3. Practical Scenarios and Examples:
To illustrate these generational differences, consider the following scenarios:
- Baby boomers may prefer a structured work environment with clear hierarchies, while millennials may thrive in a more flexible and collaborative setting.
- Baby boomers may prioritize job security and financial stability, while millennials may prioritize personal growth and work-life balance.
- Baby boomers may prefer face-to-face communication, while millennials may be more comfortable with digital communication tools.

4. Navigating Generational Differences:
To create a harmonious and productive work environment, organizations can consider the following strategies:
- Foster open communication and create opportunities for intergenerational dialogue and understanding.
- Provide flexible work arrangements and opportunities for personal growth and development.
- Recognize and appreciate the diverse perspectives and strengths that each generation brings to the table.
- Implement mentorship or reverse-mentoring programs to facilitate knowledge sharing and collaboration between different generations.
- Create a culture of inclusivity and respect, valuing the contributions of all employees regardless of their generational background.

In summary, generational differences in work expectations and values exist between baby boomers and millennials. These differences can be attributed to various factors such as socioeconomic influences, cultural factors, and life experiences. Understanding and navigating these differences is crucial for organizations to create a harmonious and productive work environment. By fostering open communication, providing flexibility, and appreciating the diverse perspectives of each generation, organizations can effectively manage generational differences and leverage the strengths of their multi-generational workforce.'
"""

In [25]:
# Input query to LLM
input_text = {
    "context": context_rag,
    "question": question_rag
}

# Generate and print LLM response
response = llm_chain.invoke(input_text)

wrapped_text = wrap_text(response)
print(f'The answer is: \n {wrapped_text}')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The answer is: 
 
### [INST]
Instruction: I will ask you a QUESTION and give you a CONTEXT and you will respond with an
answer easily understandable.

### CONTEXT:

'Generational differences in work expectations and values are a significant topic of
discussion in organizations today. Specifically, the differences between baby boomers and
millennials have garnered attention due to their coexistence in the workplace. These
generational differences can impact management dynamics at various levels, including
operational, team, and strategic levels. Understanding these differences and finding ways
to navigate them is crucial for creating a harmonious and productive work environment.

1. Contrasting Work Values, Attitudes, and Preferences:
Baby Boomers:
- Work Values: Baby boomers tend to prioritize loyalty, dedication, and hard work. They
often value stability and long-term commitment to their organizations.
- Attitudes: Baby boomers may have a more traditional approach to work, valuing hie

# Streaming Output & Metrics for Evaluation

In [26]:
from transformers import TextIteratorStreamer
from threading import Thread
import time

first_token_time = 0
token_times = []


# Initialize a TextIteratorStreamer object for streaming text generation
streamer = TextIteratorStreamer(tokenizer, timeout=10., skip_prompt=True, skip_special_tokens=True)

# Create a text generation pipeline using the Hugging Face transformers library
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.3,
    repetition_penalty=1.1,
    max_new_tokens=1000,
    do_sample=True,
    streamer=streamer  # Use the streamer for streaming text generation
)

prompt_template = """
### [INST]
Instruction: I will ask you a QUESTION and give you a CONTEXT and you will respond with an answer easily understandable.

### CONTEXT:
{context}

### QUESTION:
{question}

[/INST]
 """

mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = prompt | mistral_llm | StrOutputParser()


# Input query to LLM
input_text = {
    "context": """"
    Act as a gourmet chef.
    I have a friend coming over who is a vegetarian.
    I want to impress my friend with a special vegetarian dish.
    """,
    "question": "Give me two options, along with the whole recipe for each."
}



Device set to use cuda:0


In [27]:
# Initialize variables for time measurements
start_time = time.time()

# Start a new thread to invoke the language model chain with the input text
thread = Thread(target=llm_chain.invoke, args=[input_text])
thread.start()

# Initialize a variable to store the model output
model_output = ""

# Iterate over the streamer to get the generated text in chunks
for i, new_text in enumerate(streamer):
    model_output += new_text
    print(new_text, end='')

    # Measure time for the first token
    if i == 0:
        first_token_time = time.time()
    # Measure time for each token
    token_times.append(time.time())

# Calculate end-to-end latency
end_time = time.time()
end_to_end_latency = end_time - start_time

# Calculate time to first token
ttft = first_token_time - start_time

# Calculate inter-token latency
itl = sum(x - y for x, y in zip(token_times[1:], token_times[:-1])) / (len(token_times) - 1)

# Calculate throughput
throughput = len(tokenizer.encode(model_output)) / end_to_end_latency

print("\nTime To First Token (TTFT):", ttft)
print("Inter-token latency (ITL):", itl)
print("End-to-end Latency:", end_to_end_latency)
print("Throughput:", throughput)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1. Vegetarian Lasagna Recipe:

Ingredients:

* 8 lasagna noodles
* 1 tablespoon olive oil
* 1 onion, chopped
* 3 cloves garlic, minced
* 2 cups chopped mushrooms
* 1 cup chopped spinach
* 2 cups ricotta cheese
* 1 egg
* 1 teaspoon dried basil
* 1 teaspoon dried oregano
* 1/4 teaspoon salt
* 1/4 teaspoon black pepper
* 2 cups tomato sauce
* 1 cup shredded mozzarella cheese
* 1/4 cup grated parmesan cheese

Instructions:

1. Preheat oven to 375°F (190°C).
2. Cook lasagna noodles according to package instructions until al dente. Drain and set aside.
3. In a large skillet, heat olive oil over medium heat. Add onion and garlic, and cook until softened, about 5 minutes.
4. Add mushrooms and spinach to the skillet, and cook until the vegetables are tender, about 5 minutes.
5. In a separate bowl, mix together ricotta cheese, egg, basil, oregano, salt, and pepper.
6. Spread a layer of tomato sauce onto the bottom of a 9x13 inch baking dish. Place 4 lasagna noodles on top of the sauce. Spread 1/

Let's create it into a function

In [28]:
def streaming_inference(context, question):
    # Initialize a TextIteratorStreamer object for streaming text generation
    streamer = TextIteratorStreamer(tokenizer, timeout=10., skip_prompt=True, skip_special_tokens=True)

    # Create a text generation pipeline using the Hugging Face transformers library
    text_generation_pipeline = transformers.pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        temperature=0.3,
        repetition_penalty=1.1,
        max_new_tokens=1000,
        do_sample= True,
        streamer=streamer  # Use the streamer for streaming text generation
    )

    prompt_template = """
    ### [INST]
    Instruction: I will ask you a QUESTION and give you a CONTEXT and you will respond with an answer easily understandable in bullet points.

    ### CONTEXT:
    {context}

    ### QUESTION:
    {question}

    [/INST]
    """

    mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

    prompt = PromptTemplate(
        input_variables=["context", "question"],
        template=prompt_template,
    )

    llm_chain = prompt | mistral_llm | StrOutputParser()

    input_text = {
        "context": context,
        "question": question
    }

    # Initialize variables for time measurements
    start_time = time.time()

    # Start a new thread to invoke the language model chain with the input text
    thread = Thread(target=llm_chain.invoke, args=[input_text])
    thread.start()

    # Initialize a variable to store the model output
    model_output = ""

    # Iterate over the streamer to get the generated text in chunks
    for i, new_text in enumerate(streamer):
        model_output += new_text
        print(new_text, end='')

        # Measure time for the first token
        if i == 0:
            first_token_time = time.time()
        # Measure time for each token
        token_times.append(time.time())

    # Calculate end-to-end latency
    end_time = time.time()
    end_to_end_latency = end_time - start_time

    # Calculate time to first token
    ttft = first_token_time - start_time

    # Calculate inter-token latency
    itl = sum(x - y for x, y in zip(token_times[1:], token_times[:-1])) / (len(token_times) - 1)

    # Calculate throughput
    throughput = len(tokenizer.encode(model_output)) / end_to_end_latency

    print("\nTime To First Token (TTFT):", ttft)
    print("Inter-token latency (ITL):", itl)
    print("End-to-end Latency:", end_to_end_latency)
    print("Throughput:", throughput)

    # Return the metrics
    return {
        "TTFT": ttft,
        "ITL": itl,
        "End-to-end Latency": end_to_end_latency,
        "Throughput": throughput
    }


In [29]:
streaming_inference(context_rag,question_rag)

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



   The differences in work expectations between generations can be attributed to various factors such as socioeconomic influences, cultural factors, and life experiences. Here are some key differences between baby boomers and millennials in terms of work expectations:

   - Baby boomers tend to prioritize loyalty, dedication, and hard work. They often value stability and long-term commitment to their organizations. In contrast, millennials often prioritize work-life balance, personal growth, and purpose-driven work. They seek opportunities for learning and development.
   - Baby boomers may have a more traditional approach to work, valuing hierarchy and authority. They may prefer a structured work environment and clear guidelines. On the other hand, millennials may have a more collaborative and inclusive approach to work. They value flexibility, autonomy, and a sense of purpose in their roles.
   - Baby boomers may prefer face-to-face communication and value personal relationships in 

{'TTFT': 6.001152276992798,
 'ITL': 0.10271444575320432,
 'End-to-end Latency': 29.383474349975586,
 'Throughput': 10.55014789291783}

Time To First Token (TTFT): 5.885674238204956
Inter-token latency (ITL): 0.25101944495291606
End-to-end Latency: 29.334325551986694
Throughput: 7.124759000496103

# Setting Up a FastAPI Wrapper

In [30]:
from fastapi import FastAPI, Request, BackgroundTasks
from fastapi.responses import StreamingResponse
from threading import Thread

app = FastAPI()

# Variables for time measurements
start_time = 0
first_token_time = 0
token_times = []

# Invoke the LLM chain using the input text
def invoke_llm_chain(input_text):
    llm_chain.invoke(input_text)


# Function to calculate metrics
def calculate_metrics(start_time, first_token_time, token_times, model_output):
    end_time = time.time()
    end_to_end_latency = end_time - start_time
    ttft = first_token_time - start_time
    itl = sum(x - y for x, y in zip(token_times[1:], token_times[:-1])) / (len(token_times) - 1)

    throughput = len(model_output) / end_to_end_latency
    return {
        "End-to-end Latency": end_to_end_latency,
        "Time To First Token (TTFT)": ttft,
        "Inter-token latency (ITL)": itl,
        "Throughput": throughput
    }

# Generate output text using the streamer
def generate_output(streamer):
    global start_time, first_token_time, token_times, model_output
    model_output = ""
    start_time = time.time()

    for i, new_text in enumerate(streamer):
        model_output += new_text

        # Measure time for the first token
        if i == 0:
            first_token_time = time.time()

        # Measure time for each token
        token_times.append(time.time())
        yield new_text

    metrics = calculate_metrics(start_time, first_token_time, token_times, model_output)
    print("Metrics:", metrics)
    return metrics


@app.get("/")
async def root():
    return {"message": "Hello, World!"}


@app.post("/mistral-inference")
async def mistral_inference(input_text: dict, background_tasks: BackgroundTasks):
    # Start a separate thread to run the LLM chain asynchronously
    thread = Thread(target=invoke_llm_chain, args=[input_text])
    thread.start()

    # Add the generate_output function to the background tasks with the streamer
    background_tasks.add_task(generate_output, streamer)

    return StreamingResponse(generate_output(streamer))


# Using Ngrok to Create Public URL

Sign up for your own key on Ngrok

In [36]:
NGROK_KEY = userdata.get('NGROK_KEY')

In [37]:
!ngrok config add-authtoken $NGROK_KEY

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [38]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn
ngrok.kill()

In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()

uvicorn.run(host="127.0.0.1", port=8000, app=app)


INFO:     Started server process [463]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://dd9e-34-16-164-196.ngrok-free.app
INFO:     77.58.33.159:0 - "GET / HTTP/1.1" 200 OK
INFO:     77.58.33.159:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


INFO:     34.73.228.229:0 - "POST /mistral-inference HTTP/1.1" 200 OK
Metrics: {'End-to-end Latency': 13.393592357635498, 'Time To First Token (TTFT)': 4.418612718582153, 'Inter-token latency (ITL)': 0.0735610625782951, 'Throughput': 47.933368648031525}
INFO:     34.73.228.229:0 - "POST /mistral-inference HTTP/1.1" 200 OK


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Metrics: {'End-to-end Latency': 29.286555528640747, 'Time To First Token (TTFT)': 5.0306620597839355, 'Inter-token latency (ITL)': 0.8936832139083931, 'Throughput': 48.41812136675714}
INFO:     34.73.228.229:0 - "POST /mistral-inference HTTP/1.1" 200 OK


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Metrics: {'End-to-end Latency': 11.924010038375854, 'Time To First Token (TTFT)': 4.234210014343262, 'Inter-token latency (ITL)': 1.0117340204720056, 'Throughput': 46.293151232132544}
INFO:     34.73.228.229:0 - "POST /mistral-inference HTTP/1.1" 200 OK


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Metrics: {'End-to-end Latency': 16.09711790084839, 'Time To First Token (TTFT)': 5.013424873352051, 'Inter-token latency (ITL)': 0.8196859604971749, 'Throughput': 39.19956379065494}
